In [ ]:
!pip install langchain langchain-openai sendgrid python-dotenv


In [ ]:
from dotenv import load_dotenv
import os

load_dotenv(override=True)

SENDGRID_API_KEY = os.getenv("SENDGRID_API_KEY")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")


In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gemini-2.0-flash",
    api_key=GEMINI_API_KEY,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
    temperature=0.7
)


In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain


In [ ]:
professional_prompt = PromptTemplate(
    input_variables=["input"],
    template="""
You are a professional, serious sales agent working for ComplAI.
Write a professional cold sales email.

{input}
"""
)

engaging_prompt = PromptTemplate(
    input_variables=["input"],
    template="""
You are a humorous, engaging sales agent working for ComplAI.
Write a witty cold sales email.

{input}
"""
)

busy_prompt = PromptTemplate(
    input_variables=["input"],
    template="""
You are a busy sales agent working for ComplAI.
Write a concise cold sales email.

{input}
"""
)


In [ ]:
sales_agent1 = LLMChain(llm=llm, prompt=professional_prompt)
sales_agent2 = LLMChain(llm=llm, prompt=engaging_prompt)
sales_agent3 = LLMChain(llm=llm, prompt=busy_prompt)


In [ ]:
import asyncio

async def generate_emails(message):
    results = await asyncio.gather(
        sales_agent1.apredict(input=message),
        sales_agent2.apredict(input=message),
        sales_agent3.apredict(input=message),
    )
    return results

emails = await generate_emails("Write a cold sales email")


In [ ]:
for email in emails:
    print(email, "\n\n")


In [ ]:
picker_prompt = PromptTemplate(
    input_variables=["emails"],
    template="""
You are a customer.
Pick the single best cold sales email below.
Do NOT explain your choice.
Return ONLY the selected email.

Emails:
{emails}
"""
)

sales_picker = LLMChain(llm=llm, prompt=picker_prompt)


In [ ]:
emails_text = "\n\n---\n\n".join(emails)
best_email = await sales_picker.apredict(emails=emails_text)

print("Best Email:\n", best_email)


In [ ]:
from langchain.tools import tool
import sendgrid
from sendgrid.helpers.mail import Mail, Email, To, Content


In [ ]:
@tool
def send_email(body: str) -> str:
    """Send the selected sales email to the customer."""
    sg = sendgrid.SendGridAPIClient(api_key=SENDGRID_API_KEY)

    from_email = Email("chaitanyajawanjal2@gmail.com")
    to_email = To("sevasamruddhi@gmail.com")
    content = Content("text/plain", body)

    mail = Mail(from_email, to_email, "Sales Email", content).get()
    sg.client.mail.send.post(request_body=mail)

    return "Email sent successfully"


In [ ]:
from langchain.agents import initialize_agent, AgentType


In [ ]:
manager_prompt = """
You are a Sales Manager at ComplAI.

Steps:
1. Generate 3 drafts using sales agents.
2. Choose the best one.
3. Send exactly ONE email using send_email tool.

Never send more than one email.
"""


In [ ]:
manager_agent = initialize_agent(
    tools=[send_email],
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True
)


In [ ]:
manager_agent.invoke(
    f"""
Drafts:
{emails_text}

Best Email:
{best_email}

Send it.
"""
)
